In [48]:
using Pkg
for p in ["Knet", "Plots", "IterTools","PyCall"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end
using DelimitedFiles
using Knet: KnetArray, accuracy, progress, minibatch, cycle, adam, xavier_uniform, progress!, @save, @load
using Plots
using IterTools: ncycle, takenth, take
using Base.Iterators: flatten
using LinearAlgebra

include("utils.jl")
include("models.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [49]:
function mytrain!(model, data, epochs, lr, window_size)
    early_stop_counter = 0
    prev_val_loss = 0
    iter = 0
    
    trnloss = []
    valloss = []
    
    flag = true

    function task()     
        
        append!(trnloss, model(data))
        v_loss = val_loss(model, data)
        append!(valloss, v_loss) 
        
        if v_loss >= prev_val_loss
            early_stop_counter = early_stop_counter + 1
        else
            early_stop_counter = 0
        end
        if early_stop_counter == window_size
            flag = false
        end 
        iter = iter + 1
        prev_val_loss = v_loss
        return flag
        
    end
        
    training = adam(model, ncycle(data, epochs), lr=lr)
    progress!(flag = task() for x in (x for (i,x) in enumerate(training)) if flag)
    return 1:iter, trnloss, valloss
end

mytrain! (generic function with 1 method)

In [50]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
    window_size
end

arguments = args(200, 0.01, 5e-4, 16, 0.5, 10)
function val_loss(g::GCN,x,y)
    output = g(x)[:, idx_val]
    nll(output, y[idx_val]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
end  
function val_loss(g::GCN, d)
    mean(val_loss(g,x,y) for (x,y) in d)
end

function test_loss(g::GCN,x,y)
    output = g(x)[:, idx_test]
    nll(output, y[idx_test]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
end  
function test_loss(g::GCN,d)
    mean(test_loss(g,x,y) for (x,y) in d)
end

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 

In [51]:
#####################################################################################################################

In [52]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("cora")

(
  [1   ,    1]  =  0.25
  [634 ,    1]  =  0.25
  [1863,    1]  =  0.223607
  [2583,    1]  =  0.25
  [2   ,    2]  =  0.25
  [3   ,    2]  =  0.204124
  [653 ,    2]  =  0.288675
  [655 ,    2]  =  0.353553
  [2   ,    3]  =  0.204124
  [3   ,    3]  =  0.166667
  [333 ,    3]  =  0.166667
  [1455,    3]  =  0.288675
  ⋮
  [288 , 2706]  =  0.5
  [2706, 2706]  =  0.5
  [166 , 2707]  =  0.2
  [170 , 2707]  =  0.258199
  [1474, 2707]  =  0.2
  [2707, 2707]  =  0.2
  [2708, 2707]  =  0.2
  [166 , 2708]  =  0.2
  [599 , 2708]  =  0.0766965
  [1474, 2708]  =  0.2
  [2707, 2708]  =  0.2
  [2708, 2708]  =  0.2, 
  [20  ,    1]  =  0.00178571
  [82  ,    1]  =  0.0222222
  [147 ,    1]  =  0.0625
  [316 ,    1]  =  0.0526316
  [775 ,    1]  =  0.00248756
  [878 ,    1]  =  0.016129
  [1195,    1]  =  0.0232558
  [1248,    1]  =  0.00917431
  [1275,    1]  =  0.00625
  [20  ,    2]  =  0.00178571
  [89  ,    2]  =  0.0133333
  [150 ,    2]  =  0.025641
  ⋮
  [187 , 2708]  =  0.04
  [330 , 270

In [53]:
features

1433×2708 SparseMatrixCSC{Float32,Int64} with 49216 stored entries:
  [20  ,    1]  =  0.00178571
  [82  ,    1]  =  0.0222222
  [147 ,    1]  =  0.0625
  [316 ,    1]  =  0.0526316
  [775 ,    1]  =  0.00248756
  [878 ,    1]  =  0.016129
  [1195,    1]  =  0.0232558
  [1248,    1]  =  0.00917431
  [1275,    1]  =  0.00625
  [20  ,    2]  =  0.00178571
  [89  ,    2]  =  0.0133333
  [150 ,    2]  =  0.025641
  ⋮
  [187 , 2708]  =  0.04
  [330 , 2708]  =  0.0526316
  [448 , 2708]  =  0.0169492
  [455 , 2708]  =  0.0769231
  [755 , 2708]  =  0.00917431
  [775 , 2708]  =  0.00248756
  [897 , 2708]  =  0.0526316
  [1023, 2708]  =  0.0434783
  [1115, 2708]  =  0.0169492
  [1329, 2708]  =  0.00271739
  [1413, 2708]  =  0.025641
  [1415, 2708]  =  0.025641

In [54]:
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

GCN(GCLayer(P(Array{Float64,2}(16,1433)), P(Array{Float32,1}(16)), 
  [1   ,    1]  =  0.25
  [634 ,    1]  =  0.25
  [1863,    1]  =  0.223607
  [2583,    1]  =  0.25
  [2   ,    2]  =  0.25
  [3   ,    2]  =  0.204124
  [653 ,    2]  =  0.288675
  [655 ,    2]  =  0.353553
  [2   ,    3]  =  0.204124
  [3   ,    3]  =  0.166667
  [333 ,    3]  =  0.166667
  [1455,    3]  =  0.288675
  ⋮
  [288 , 2706]  =  0.5
  [2706, 2706]  =  0.5
  [166 , 2707]  =  0.2
  [170 , 2707]  =  0.258199
  [1474, 2707]  =  0.2
  [2707, 2707]  =  0.2
  [2708, 2707]  =  0.2
  [166 , 2708]  =  0.2
  [599 , 2708]  =  0.0766965
  [1474, 2708]  =  0.2
  [2707, 2708]  =  0.2
  [2708, 2708]  =  0.2, NNlib.relu), GCLayer(P(Array{Float64,2}(7,16)), P(Array{Float32,1}(7)), 
  [1   ,    1]  =  0.25
  [634 ,    1]  =  0.25
  [1863,    1]  =  0.223607
  [2583,    1]  =  0.25
  [2   ,    2]  =  0.25
  [3   ,    2]  =  0.204124
  [653 ,    2]  =  0.288675
  [655 ,    2]  =  0.353553
  [2   ,    3]  =  0.204124
  [3   ,   

In [55]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

2708-element Array{Int64,1}:
 4
 5
 5
 1
 4
 3
 1
 4
 4
 3
 1
 1
 5
 ⋮
 4
 4
 4
 4
 4
 4
 1
 4
 4
 4
 4
 4

In [56]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded))

Data{Tuple{SparseMatrixCSC{Float32,Int64},Array{Int64,1}}}(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [4 5 … 4 4], 2708, 2708, false, 1, 1:2708, false, (1433, 2708), (2708,), SparseMatrixCSC{Float32,Int64}, Array{Int64,1})

In [ ]:
iters, trnloss, vallos = mytrain!(model, data, arguments.epochs, arguments.lr, arguments.window_size)

[49, 04:06, 5.64s/i]  

In [ ]:
plot(iters, [trnloss, vallos] , xlim=(1:3),labels=[:trn :val :tst], xlabel="epochs", ylabel="loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
accuracy(output[:,idx_test], labels_decoded[idx_test])

In [ ]:
png("cora")

In [45]:
#####################################################################################################################

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("citeseer")

In [ ]:
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

In [ ]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

In [ ]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded))

In [ ]:
iters, trnloss, vallos = mytrain!(model, data, arguments.epochs, arguments.lr, arguments.window_size)

In [ ]:
plot(iters, [trnloss, vallos] , xlim=(1:3),labels=[:trn :val :tst], xlabel="epochs", ylabel="loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
accuracy(output[:,idx_test], labels_decoded[idx_test])

In [ ]:
png("citeseer")

In [ ]:
#####################################################################################################################

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("pubmed")

In [ ]:
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

In [ ]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

In [ ]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded))

In [ ]:
iters, trnloss, vallos = mytrain!(model, data, arguments.epochs, arguments.lr, arguments.window_size)

In [ ]:
plot(iters, [trnloss, vallos] , xlim=(1:3),labels=[:trn :val :tst], xlabel="epochs", ylabel="loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
accuracy(output[:,idx_test], labels_decoded[idx_test])

In [ ]:
png("pubmed")